In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score,f1_score,auc,roc_curve,roc_auc_score, ConfusionMatrixDisplay
from sklearn.metrics import classification_report


In [10]:
df= pd.read_csv('cleaned_df.csv')
df.head()

,Unnamed: 0,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,...,Dst_Port_Category,Pkt_Len_Var,Pkt_Len_Median,Fwd_Entropy,Flag_Combination,Source_IP_Private,Destination_IP_Private,Src_IP_Frequency,Dst_IP_Frequency,Is_Inbound
0,0,51023,443,1,151054,6,8,1076.0,4575.0,821.0,...,2,178204.5,1119.5,2.0,1,1,0,177977,344,0
1,1,51023,443,1,349,2,0,23.0,0.0,23.0,...,2,264.5,11.5,1.0,2,1,0,177977,344,0
2,2,34259,443,1,119,2,0,23.0,0.0,23.0,...,2,264.5,11.5,1.0,2,1,0,177977,882,0
3,3,55509,443,1,37055,1,1,31.0,0.0,31.0,...,2,480.5,15.5,1.0,2,1,0,177977,575,0
4,4,44852,443,1,178727,6,7,1313.0,307.0,753.0,...,2,171112.5,460.5,1.0,1,1,0,177977,344,0


In [11]:
label_encoder = LabelEncoder()
df['Label'] = label_encoder.fit_transform(df['Label'])


In [12]:
X = df.drop('Label', axis=1)
y = df['Label']

In [13]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [14]:
y_encoded = to_categorical(y)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

In [18]:
X.shape


(392034, 96)

In [16]:
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))


In [17]:
X_train.shape

(313627, 1, 96)

In [19]:
ltsm = Sequential()
ltsm.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), activation='relu'))
ltsm.add(Dropout(0.2))
ltsm.add(Dense(32, activation='relu'))
ltsm.add(Dense(y_train.shape[1], activation='softmax'))


c:\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [20]:
ltsm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [21]:
history = ltsm.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test), verbose=2)

Epoch 1/20
4901/4901 - 14s - 3ms/step - accuracy: 0.6472 - loss: 0.8705 - val_accuracy: 0.7496 - val_loss: 0.5341
Epoch 2/20
4901/4901 - 11s - 2ms/step - accuracy: 0.7514 - loss: 0.5342 - val_accuracy: 0.7914 - val_loss: 0.4349
Epoch 3/20
4901/4901 - 12s - 2ms/step - accuracy: 0.7727 - loss: 0.4692 - val_accuracy: 0.7996 - val_loss: 0.3980
Epoch 4/20
4901/4901 - 12s - 2ms/step - accuracy: 0.7847 - loss: 0.4325 - val_accuracy: 0.8098 - val_loss: 0.3707
Epoch 5/20
4901/4901 - 11s - 2ms/step - accuracy: 0.7925 - loss: 0.4113 - val_accuracy: 0.8188 - val_loss: 0.3545
Epoch 6/20
4901/4901 - 11s - 2ms/step - accuracy: 0.7971 - loss: 0.3980 - val_accuracy: 0.8179 - val_loss: 0.3430
Epoch 7/20
4901/4901 - 12s - 2ms/step - accuracy: 0.8007 - loss: 0.3892 - val_accuracy: 0.8221 - val_loss: 0.3379
Epoch 8/20
4901/4901 - 12s - 2ms/step - accuracy: 0.8041 - loss: 0.3809 - val_accuracy: 0.8261 - val_loss: 0.3320
Epoch 9/20
4901/4901 - 12s - 2ms/step - accuracy: 0.8065 - loss: 0.3739 - val_accuracy: 

In [23]:
results = ltsm.evaluate(X_test, y_test, verbose=2)
y_pred_prob = ltsm.predict(X_test)  # Get the predicted probabilities
y_pred = (y_pred_prob > 0.5).astype(int)  

2451/2451 - 3s - 1ms/step - accuracy: 0.8334 - loss: 0.3082
2451/2451 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step


In [28]:
f1 = f1_score(y_test, y_pred, average='weighted')
roc_auc = roc_auc_score(y_test, y_pred_prob, multi_class='ovr', average='weighted')

print(f"Restored Model Test Loss: {results[0]}")
print(f"Restored Model Test Accuracy: {results[1]}")

print(f"F1 Score: {f1}")
print(f"ROC AUC Score: {roc_auc}")

Restored Model Test Loss: 0.30824461579322815
Restored Model Test Accuracy: 0.833433210849762
F1 Score: 0.8291553892585495
ROC AUC Score: 0.9900731498558166
